# Face Recognition

- face verification : 辨識是某個人
- face recognition : 辨識是 K 人中的哪一個

One-Shot learning : 訓練辨識 是否為甲。缺點: 沒有很多甲的照片，要加入新人時候，需要重新訓練。

Learning a "similarity" function:

$ d \big( img1, img2 \big) $ = degree of difference between images.

If $ d \big( img1, img2 \big) \le \tau $, recognize as "same" person.

## Siamese Network

用同樣的 CNN ，最終 FC 輸出 128 個 feature 值 : $ f\big( x \big) $

訓練時候使用兩個不同的照片當成輸入，比較 

$$
d \Big( x^{(1)}, x^{(1)} \Big) = \Big\Vert \ f\big( x^{(1)} \big) - f\big( x^{(2)} \big)  \ \Big\Vert^2_2
$$

## Triplet Loss

given 3 images:

- A : Anchor
- P : Positive
- N : Negative


希望 A,P 較像，數值較小；alpha 是 margin，避免直接用0滿足

$$
\begin{cases}
d\big(A,P\big) + \alpha \ \ & \le \ \  d\big(A,P\big) \\
d\big(A,P\big) - d\big(A,P\big)  + \alpha \ \ & \le \ \  0
\end{cases}
$$

$$
\begin{cases}
\mathcal{L} \big( A, P, N \big) & = 
\max \Big( 
\big\Vert \ f(A) - f(P) \ \big\Vert_2^2 - 
\big\Vert \ f(A) - f(N) \ \big\Vert_2^2 + \alpha,\ \ \  0
\Big) \\
\mathcal{J} & = \sum_i^m \mathcal{L} \big( A^{(i)}, P^{(i)}, N^{(i)} \big)
\end{cases}
$$

需要慎選 A,P,N 組合，如果隨機，訓練集太容易，則上式 J 容易滿足，訓練沒有效果。

## Face verification and binary classification

$$
\hat{y} = \sigma \Big( 
\sum_{k=1}^{128} w_i \big| \ 
f(x^{(i)})_k - f(x^{(j)})_k
\ \big| \ + b
\Big)
$$

or using Chi-Square

$$
\chi^2 = \frac{\big( f(x^{(i)})_k - f(x^{(j)})_k \big)^2}{f(x^{(i)})_k + f(x^{(j)})_k}
$$


# Neural Style Transfer

## Cost Function

Given 2 images: 

- C: Content Image
- S: Style Image

Output Generated Image: G

$$
\mathcal{J} \big( G \big) = 
\alpha \ \mathcal{J}_{\text{content}} \big( C, G \big) +
\beta  \ \mathcal{J}_{\text{style}} \big( S, G \big)
$$

Process:

1. Initial G randomly, such as G: 100x100x3

2. Use gradient descent to minimize $ \mathcal{J} \big( G \big) $

$$
G := G - \frac{\partial}{\partial \ G} \mathcal{J} \big( G \big)
$$

## Content Cost Function

- 用一個 hidden layer: l 計算 content cost, 通常是 CNN 中間的層，不太深不太淺。
- 使用一個已經 預訓練 好的 CNN 如 VGG network
- let $ a^{[l](G)}, a^{[l](C)} $ be activations of layer l on the images.
- 如果 $ a^{[l](G)}, a^{[l](C)} $ 相似，則 C,G 內容相似。

$$
\mathcal{J}_{\text{content}} \big( G \big) = \frac{1}{2}
\big\Vert a^{[l](G)} - a^{[l](C)} \big\Vert^2
$$

## Style Cost Function

在選出的 hidden layer l 上，比較每個 channel 的 correlation (Un-normalized cross co-variance).

如 l 的 dimension : $ n_h, n_w, n_c $

$ n_c $ 是每個不同的 channel 數量。

Let 
$  
a^{[l]}_{i,j,k} = 
$ activation at (i, j, k), 

$
G^{[l]}
$ be a $ n_c \times n_c $ matrix of correleations

$ G^{[l]}_{k, k'} $ : 在 k channel 和 k' channel 的 correlation.

$$
G^{[l]}_{k, k'} = \sum_i^{n_H} \sum_j^{n_W} a^{[l]}_{i,j,k} \times a^{[l]}_{i,j,k'}
$$

上式就是Gram Matrix, 算出 S, G 的 Gram Matrix, 就可得到 J_style

$$
\begin{cases}
G^{[l](S)}_{k, k'} & = \sum_i^{n_H} \sum_j^{n_W} a^{[l](S)}_{i,j,k} \times a^{[l](S)}_{i,j,k'} \\
G^{[l](G)}_{k, k'} & = \sum_i^{n_H} \sum_j^{n_W} a^{[l](G)}_{i,j,k} \times a^{[l](G)}_{i,j,k'}
\\
\mathcal{J}_{\text{style}}^{[l]} \big( S, G \big) & = 
\frac{1}{(2 n^{[l]}_h n^{[l]}_w n^{[l]}_c)^2} \ 
\Big\Vert
G^{[l](S)}_{k, k'} - G^{[l](G)}_{k, k'}
\Big\Vert_F^2 \\
& = 
\frac{1}{(2 n^{[l]}_h n^{[l]}_w n^{[l]}_c)^2} \ 
\sum_{k} \sum_{k'} \Big( G^{[l](S)}_{k, k'} - G^{[l](G)}_{k, k'} \Big)^2
\end{cases}
$$

通常使用數個 layers, 淺的 + 深的，會獲得較好的視覺效果。

$$
\mathcal{J}_{\text{style}} \Big( S, G \Big) =
\sum_l \lambda^{[l]} \mathcal{J}_{\text{style}}^{[l]} \big( S, G \big)
$$

## 1D, 3D generation